In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
 import torch
 import torch.nn as nn
 import torch.optim as optim
 from torchvision import datasets,transforms,models
 from torch.utils.data import DataLoader,random_split,Subset
 import matplotlib.pyplot as plt
 from PIL import ImageFile
 import os
 from PIL import Image
 ImageFile.LOAD_TRUNCATED_IMAGES=True

In [3]:
transform=transforms.Compose([transforms.Resize((128,128)),transforms.ToTensor(),transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])])

In [5]:
data_dir='/content/drive/MyDrive/PetImagesSampled'

dataset = datasets.ImageFolder(data_dir, transform=transform)

train_size=int(0.7*len(dataset))
val_size=int(0.15*len(dataset))
test_size=len(dataset)-train_size-val_size
train_dataset,val_dataset,test_dataset=random_split(dataset,[train_size,val_size,test_size])

batch_size=64
train_loader=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
val_loader=DataLoader(val_dataset,batch_size=batch_size,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=batch_size,shuffle=True)

In [6]:
class SimpleCNN(nn.Module):
  def __init__(self,n_classes):
    super(SimpleCNN,self).__init__()
    self.conv1=nn.Conv2d(3,16,kernel_size=3,padding=1)
    self.pool=nn.MaxPool2d(2,2)
    self.conv2=nn.Conv2d(16,32,kernel_size=3,padding=1)
    self.fc1=nn.Linear(32*7*7,500)
    self.fc2=nn.Linear(500,n_classes)
    self.relu=nn.ReLU()
    self.adaptive_pool=nn.AdaptiveAvgPool2d((7,7))
  def forward(self,x):
    x=self.relu(self.conv1(x))
    x=self.pool(x)
    x=self.relu(self.conv2(x))
    x=self.pool(x)
    x=self.adaptive_pool(x)
    x=x.view(-1,32*7*7)
    x=self.relu(self.fc1(x))
    x=self.fc2(x)
    return x

In [7]:
from torchvision.models import mobilenet_v2,MobileNet_V2_Weights
weights=MobileNet_V2_Weights.DEFAULT
mobilenet=mobilenet_v2(weights=weights)
mobilenet.classifier[1]=nn.Linear(mobilenet.last_channel,2)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-7ebf99e0.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 60.2MB/s]


In [12]:
def train(model,optimizer,criterion,train_loader,val_loader,num_epochs=5):
  model=model.to(device)
  for epoch in range(num_epochs):
    model.train()
    running_loss=0.0
    for inputs,labels in train_loader:
      inputs,labels=inputs.to(device),labels.to(device)
      optimizer.zero_grad()
      outputs=model(inputs)
      loss=criterion(outputs,labels)
      loss.backward()
      optimizer.step()
      running_loss+=loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}],Loss:{running_loss/len(train_loader):.4f}')
    evaluate(model,val_loader)
def evaluate(model,loader):
  model.eval()
  correct=0
  total=0
  with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
  print(f'Accuracy: {100 * correct / total:.2f}%')


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

simple_cnn = SimpleCNN(n_classes=2).to(device)
optimizer = optim.Adam(simple_cnn.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
print("Training SimpleCNN:")
train(simple_cnn, optimizer, criterion, train_loader, val_loader, num_epochs=5)
print("\nEvaluating SimpleCNN on test set:")
evaluate(simple_cnn, test_loader)

mobilenet = mobilenet.to(device)
optimizer_mobilenet = optim.Adam(mobilenet.parameters(), lr=0.001)
print("\nTraining MobileNetV2:")
train(mobilenet, optimizer_mobilenet, criterion, train_loader, val_loader, num_epochs=5)
print("\nEvaluating MobileNetV2 on test set:")
evaluate(mobilenet, test_loader)

Training SimpleCNN:
Epoch [1/5],Loss:0.6889
Accuracy: 63.93%
Epoch [2/5],Loss:0.6323
Accuracy: 67.32%
Epoch [3/5],Loss:0.5820
Accuracy: 68.96%
Epoch [4/5],Loss:0.5402
Accuracy: 72.68%
Epoch [5/5],Loss:0.5086
Accuracy: 74.75%

Evaluating SimpleCNN on test set:
Accuracy: 74.15%

Training MobileNetV2:
Epoch [1/5],Loss:0.0487
Accuracy: 94.86%
Epoch [2/5],Loss:0.0493
Accuracy: 97.05%
Epoch [3/5],Loss:0.0283
Accuracy: 97.16%
Epoch [4/5],Loss:0.0176
Accuracy: 96.94%
Epoch [5/5],Loss:0.0257
Accuracy: 94.21%

Evaluating MobileNetV2 on test set:
Accuracy: 94.11%
